# Introdução:


---


A implementação do último projeto se encontra abaixo.

Vale ressaltar que na minha implementação do algorítmo de busca exaustiva eu usei duas maneiras de calcular o score de alinhamento de duas subsequencias.

A primeira é no caso das subsequencias tenham tamahos distintos, o score é dado pelo resultado do algoritmo de smith-waterman. Já a segunda é caso as subsequencias tenham mesmo tamanho, nessa situação uma função w devolve 2 em match e -1 no contrário.

Como proposto no roteiro foi realizada a tentativa de implementar essas duas maneiras de calcular o score na GPU. Começando pela função w quando as seubsequencias tem o mesmo tamanho.

In [1]:
%%writefile busca_exaustiva.cu
#include <chrono>
#include <vector>
#include <iostream>
#include <thrust/transform.h>
#include <thrust/functional.h>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
using namespace std;

struct nucleotidio
{
    int id;
    char base;
};

struct mat_ele
{
    int i = -1, j = -1, val = 0;
};

struct result
{
    int score = 0;
    thrust::device_vector<nucleotidio> subseq_a;
    thrust::device_vector<nucleotidio> subseq_b;
};

// Function to create all the power set
vector<vector<nucleotidio>> createPowerSet(vector<nucleotidio> set, int set_size)
{
    vector<vector<nucleotidio>> power_set_list;
    vector<nucleotidio> subseq;

    // Set_size of power set of a set with set_size
    // n is (2^n-1)
    int pow_set_size = pow(2, set_size);
    int counter, j;

    // Run from counter 000..0 to 111..1
    for (counter = 0; counter < pow_set_size; counter++)
    {
        for (j = 0; j < set_size; j++)
        {
            // Check if jth bit in the counter is set
            // If set then save jth element from set
            if (counter & (1 << j))
            {
                nucleotidio n;
                n.id = j;
                n.base = set[j].base;
                subseq.push_back(n);
            }
        }

        if (subseq.size() > 0)
        {
            // verify if the elements in the subsequence are subseq of the original set
            bool is_subseq = true;
            int size = subseq.size();

            for (int i = 0; i < size - 1; i++)
            {
                if (subseq[i + 1].id == subseq[i].id + 1)
                {
                    is_subseq = true;
                }
                else
                {
                    is_subseq = false;
                    break;
                }
            }

            if (is_subseq)
            {
                power_set_list.push_back(subseq);
            }

            subseq.clear();
        }
    }

    return power_set_list;
}

int char_score(char a, char b)
{
    if (a == b && b != '-')
    {
        return 2;
    }
    else
    {
        return -1;
    }
}

// Função que calcula o score de duas subsequencias de mesmo tamanho paralelizando na GPU
template <typename T>
struct w
{
    int value;
    typedef int result_type;
    typedef T first_argument_type;
    typedef T second_argument_type;

    __host__ __device__
        T
        operator()(const T &x, const T &y) const
    {
        value = 0;

        if (x.base == y.base && y.base != '-')
        {
            value += 2;
        }
        else
        {
            value -= 1;
        }

        return value;
    }
};

// Algoritmo de smith-waterman para calcular o score de subsequencias com diferentes tamanhos
// int Smith_Waterman(vector<nucleotidio> seq_a, vector<nucleotidio> seq_b)
// {
//     int n = seq_a.size();
//     int m = seq_b.size();

//     vector<vector<mat_ele>> H(n + 1, vector<mat_ele>(m + 1));

//     for (int i = 0; i < n + 1; i++)
//     {
//         for (int j = 0; j < m + 1; j++)
//         {
//             mat_ele item;

//             if (j == 0 || i == 0)
//             {
//                 H[i][j] = item;
//             }
//             else
//             {

//                 int diagonal = H[i - 1][j - 1].val + char_score(seq_a[i - 1].base, seq_b[j - 1].base);
//                 int delecao = H[i - 1][j].val - 1;
//                 int insercao = H[i][j - 1].val - 1;

//                 if (max({0, diagonal, delecao, insercao}) == diagonal)
//                 {
//                     item.i = i - 1;
//                     item.j = j - 1;
//                 }
//                 else if (max({0, diagonal, delecao, insercao}) == delecao)
//                 {
//                     item.i = i - 1;
//                     item.j = j;
//                 }
//                 else if (max({0, diagonal, delecao, insercao}) == insercao)
//                 {
//                     item.i = i;
//                     item.j = j - 1;
//                 }

//                 item.val = max({0, diagonal, delecao, insercao});

//                 H[i][j] = item;
//             }
//         }
//     }

//     mat_ele max;

//     for (auto &el1 : H)
//     {
//         for (auto &el2 : el1)
//         {
//             if (el2.val > max.val)
//             {
//                 max = el2;
//             }
//         }
//     }

//     return max.val;
// }

int main()
{
    int n = 0;
    int m = 0;

    srand(24); // "Random" Seed

    cin >> n >> m;

    nucleotidio element;
    result resultado;

    // Sequencias A e B
    vector<nucleotidio> a;
    vector<nucleotidio> b;

    // Listas com todos os subconjuntos de A e B
    vector<vector<nucleotidio>> powerset_a;
    vector<vector<nucleotidio>> powerset_b;

    // Captura os elementos da primeira sequencia
    for (int i = 0; i < n; i++)
    {
        element.id = i;
        cin >> element.base;

        a.push_back(element);
    }

    // Captura os elementos da segunda sequencia
    for (int i = 0; i < m; i++)
    {
        element.id = i;
        cin >> element.base;

        b.push_back(element);
    }

    // Garantindo que a sequencia A seja a maior que a B
    // Para que as subsequencias de tamanho k não sejam maior que a sequencia a
    if (m > n)
    {
        int aux = n;
        vector<nucleotidio> temp = a;

        a = b;
        b = temp;

        n = m;
        m = aux;
    }

    // Enviando para a GPU as sequencias A e B
    thrust::device_vector<nucleotidio> gpu_a(a);
    thrust::device_vector<nucleotidio> gpu_b(b);

    // Gera todos os subconjuntos de A e B e armazena em powerset_a e powerset_b
    powerset_a = createPowerSet(a, n);
    powerset_b = createPowerSet(b, m);

    // Enviando para a GPU os powersets de A e B
    thrust::device_vector<thrust::device_vector<nucleotidio>> gpu_powerset_a(powerset_a);
    thrust::device_vector<thrust::device_vector<nucleotidio>> gpu_powerset_b(powerset_b);

    // Calcula o score de todas as subsequencias de A e B
    for (int i = 0; i < n; i++)
    {
        for (int j = 0; j < m; j++)
        {
            int temp_score = 0;

            if (gpu_powerset_a[i].size() == gpu_powerset_b[j].size())
            {
                thrust::device_vector<nucleotidio> res(gpu_powerset_b[j].size());
                thrust::transform(gpu_powerset_b[j].begin(), gpu_powerset_b[j].end(), gpu_powerset_a[i].end(), res.begin(), w());

                temp_score = thrust::reduce(res.begin(), res.end(), 0);
            }
            else
            {
                // temp_score = Smith_Waterman(gpu_powerset_a[i], gpu_powerset_b[j]);
            }

            // Verifica se o temp_score é maior que o score do resultado
            if (temp_score > resultado.score)
            {
                resultado.score = temp_score;
                resultado.subseq_a = gpu_powerset_a[i];
                resultado.subseq_b = gpu_powerset_b[j];
            }
        }
    }

    // Imprime o resultado e as subsequencias
    cout << "subsequencia A: ";
    for (int i = 0; i < resultado.subseq_a.size(); i++)
    {
        cout << resultado.subseq_a[i].base;
    }
    cout << endl;

    cout << "subsequencia B: ";
    for (int i = 0; i < resultado.subseq_b.size(); i++)
    {
        cout << resultado.subseq_b[i].base;
    }
    cout << endl;

    cout << "Score: " << resultado.score << endl;

    return 0;
}


Overwriting busca_exaustiva.cu


# Compilação e Execução

Ao executar a célula acima o arquivo `busca_exaustiva.cu` é criado e há duas formas de compilar ele.

A primeira é pelo próprio colab rodando o seguinte comando : `!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o busca_exaustiva_gpu busca_exaustiva.cu`.

A segunda pode ser realizada via CLI no terminal do monstrão com o comando: `nvcc -arch=sm_70 -std=c++14 busca_exaustiva.cu -o busca_exaustiva_gpu`.

Tendo o executável `busca_exaustiva_gpu` gerado basta passar o arquivo de input `dna_seq.txt` para a obtenção dos resultados.

In [ ]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o busca_exaustiva busca_exaustiva.cu

# Conclusão e Resultados

Para comparar os resultados e possível speedup do código com a utilização da GPU há dois arquivos de entrada na pasta do projeto, o `dna_seq.txt` e `dna_seq_2.txt`.

O primeiro arquivo possui uma entrada pequena, dessa forma acredita-se que não há significativo incremento no speedup do código com a GPU em relação ao código sequencial. Tal resultado também é constatado na implementação da busca local via OpenMP.

Já o tamanho da segunda entrada é substancialmente maior, dessa maneira acredita-se que há vantagem em no speedup do código implementado com GPU.

Essa vantagem pode ser verificada pelo tempo de processamento dos executáveis `busca_exaustiva` e `busca_exaustiva_gpu` relativos aos scripts sequenciais e paralelizados.

O tempo pode ser calculado por duas maneiras, a primeira seria adicionando a biblioteca chrono (`#include <chrono>`) e verificar o delta tempo de execução do main nos dois scripts com a função `chrono::high_resolution_clock::now()`.

E a segunda poderia ser calculada pelo próprio colab ao comparar os tempos de execução dos arquivos sequencias e paralelos usando a seguinte linha de comando: `!time ./busca_exaustiva` `!time ./busca_exaustiva_gpu`.

In [7]:
!time ./busca_exaustiva_gpu

/bin/bash: ./busca_exaustiva_gpu: No such file or directory

real	0m0.001s
user	0m0.001s
sys	0m0.000s


# Considerações finais

Durante a implementação da paraleização do script da busca exaustiva levei em consideração o que eu havia feito no projeto 3 e tentei adaptar o código para que funcionasse.

Contudo, na minha implementação do projeto 3 eu utilizei matrizes para fazer o calculo dos scores. Essas matrizes 2D eram basicamente vetores de vetores de subsequencias, ou seja, as subsequencias de A e B ficavam armazenadas dentro de um vetor e cada subsequencia consiste num vetor de nucleotídios. E essas matrizes 2D contendo as subsequencias eram passadas para as funções de cáculo de score.

O problema que enfrentei foi enviar essas essas matrizes para a GPU. O funtor `w` que criei, que se baseia na função de calculo de score paralelo também `w`, compila sem nenhum problemas. Mas, na parte do código que as matrizes são acesssadas surge erros de compilação.

Não consegui solucionar esses erros a tempo da entrega, porém, resolvendo esses problemas o próximo passo seria paralelizar a função de smith-waterman para ter a a implementação da busca exaustiva completamente paralelizada na GPU.

E depois das provas pretendo solucionar esses problemas refatorando completamento o código inserindo os pontos comentados anteriormente.